In [1]:
import geopandas as gpd
import pandas as pd
import re
import plotly.express as px

from shapely.wkt import loads
from glob import glob
from plotly.offline import plot
from shapely.geometry import Point


In [2]:
# 행정동 정보 
#files = glob("*.shp")
#shp = pd.concat([gpd.read_file(file,encoding = 'cp949') for file in files]).reset_index(drop = True)
df = pd.read_csv("../data/shp/sample.csv")
df = df[df.columns[:3]].dropna()

In [4]:
megas = df.시도명.apply(lambda x: x.replace(" ",""))
sggs= df.시군구명.apply(lambda x: x.replace(" ",''))

AttributeError: 'DataFrame' object has no attribute '시군구명'

# 중심점 계산 후 리스트로 저장 

In [180]:
centroid = [shp[((shp.MEGA_NM.apply(lambda x: re.search(mega,x)).isna()==False)&(shp.SIG_KOR_NM.apply(lambda x: re.search(sgg,x)).isna()==False))].centroid.tolist()[0] for mega,sgg in zip(megas,sggs)]

df['centroid'] = centroid

In [266]:
df.head(1)

,Name,시도명,시군구명,centroid,geometry,row_num,x,y
1,XXX,서울시,양천구,POINT (943049.1483472434 1947471.644039042),POINT (943049.148 1947471.644),1,943049.148347,1.947472e+06


In [184]:
df.to_csv("당신이 사는곳을.csv")

In [ ]:
# GeoPandas 변환

In [196]:
df.centroid = df.centroid.apply(lambda x: loads(str(x)))

In [203]:
gdf = gpd.GeoDataFrame(df,geometry = centroid,crs = 5179)

In [ ]:
# 겹치는 지역에 대한 처리 

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from plotly import graph_objects as go
from plotly import express as px
from shapely import wkt
import json
import chart_studio
import chart_studio.plotly as py


# data setup
data = pd.read_csv("/Users/jongwon/git/yourmap/result/sgg_bind.txt", sep = "|")
data['wkt'] = data.geometry.apply(wkt.loads)

gdf = gpd.GeoDataFrame(data, geometry = 'wkt')
gdf['ptr_wkt'] = gdf.centroid

gdf2 = gdf[['sgg_cd', 'sgg_nm','ptr_wkt']].copy()
gdf2 = gpd.GeoDataFrame(gdf2, geometry = 'ptr_wkt')
gdf2 = gdf2.set_crs(epsg = 5179)
gdf2 = gdf2.to_crs(epsg = 4326)

In [2]:
res = gdf2.copy()
res['lon'] = res.geometry.x
res['lat'] = res.geometry.y

In [3]:
res

,sgg_cd,sgg_nm,ptr_wkt,lon,lat
0,41111,수원시 장안구,POINT (127.00345 37.31397),127.003452,37.313966
1,41113,수원시 권선구,POINT (126.97976 37.26052),126.979758,37.260525
2,41115,수원시 팔달구,POINT (127.01624 37.27746),127.016239,37.277464
3,41117,수원시 영통구,POINT (127.05673 37.27502),127.056728,37.275019
4,41131,성남시 수정구,POINT (127.10415 37.43517),127.104145,37.435166
...,...,...,...,...,...
72,11620,관악구,POINT (126.94534 37.46738),126.945337,37.467376
73,11650,서초구,POINT (127.03122 37.47330),127.031220,37.473295
74,11680,강남구,POINT (127.06299 37.49664),127.062985,37.496644
75,11710,송파구,POINT (127.11530 37.50562),127.115295,37.505619


In [4]:
fig = px.scatter_mapbox(res,lat = 'lat',lon = 'lon',hover_name = 'sgg_nm',mapbox_style = 'open-street-map')

In [5]:
import nbformat
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

In [6]:
chart_studio.tools.set_credentials_file(username='niceguy1575', api_key='')
chart = py.plot(fig, filename = 'sgg_test_0608', auto_open = False, fileopt = 'overwrite', sharing = 'public')

In [ ]:
# 좌표변환 

In [247]:
gdf2 = gdf2.to_crs(4326)

In [249]:
gdf2['lon'] = gdf2.geometry.x
gdf2['lat'] = gdf2.geometry.y

In [267]:
fig = px.scatter_mapbox(gdf2,lat = 'lat',lon = 'lon',hover_name = 'Name',mapbox_style = 'open-street-map')

In [264]:
plot(fig, filename='notion_update2.html')

'notion_update2.html'

In [8]:
help(py.get_config)

Help on function get_config in module chart_studio.config:

get_config()
    Returns either module config or file config.



In [13]:
help(chart_studio.config.tools.set_config_file)

Help on function set_config_file in module chart_studio.tools:

set_config_file(plotly_domain=None, plotly_streaming_domain=None, plotly_api_domain=None, plotly_ssl_verification=None, plotly_proxy_authorization=None, world_readable=None, sharing=None, auto_open=None)
    Set the keyword-value pairs in `~/.plotly/.config`.
    
    :param (str) plotly_domain: ex - https://plotly.com
    :param (str) plotly_streaming_domain: ex - stream.plotly.com
    :param (str) plotly_api_domain: ex - https://api.plotly.com
    :param (bool) plotly_ssl_verification: True = verify, False = don't verify
    :param (bool) plotly_proxy_authorization: True = use plotly proxy auth creds
    :param (bool) world_readable: True = public, False = private



In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from plotly import graph_objects as go
from plotly import express as px
from shapely import wkt
import json
import chart_studio
import chart_studio.plotly as py
from shapely.geometry import Point

# data setup
data = pd.read_csv("/Users/jongwon/git/yourmap/result/sgg_bind.txt", sep = "|")
data['wkt'] = data.geometry.apply(wkt.loads)

gdf = gpd.GeoDataFrame(data, geometry = 'wkt')
gdf['ptr_wkt'] = gdf.centroid

gdf2 = gdf[['sgg_cd', 'sgg_nm','ptr_wkt']].copy()
gdf2 = gpd.GeoDataFrame(gdf2, geometry = 'ptr_wkt')
gdf2 = gdf2.set_crs(epsg = 5179)
gdf2 = gdf2.to_crs(epsg = 4326)

# json processing
res = gdf2.copy()
res['lon'] = res.geometry.x
res['lat'] = res.geometry.y

config = dict({
    'scrollZoom': True,
    'displayModeBar': True,
    'editable': True
})

fig = px.scatter_mapbox(res,lat = 'lat',lon = 'lon',hover_name = 'sgg_nm',mapbox_style = 'open-street-map')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show(config=config)

# chart-studio
# API ID: niceguy1575
# API PW: ca9QST3zpIW8HPGxoi16
chart_studio.tools.set_credentials_file(username='niceguy1575', api_key='')
chart = py.plot(fig, filename = 'sgg_test_0609', auto_open = False, fileopt = 'overwrite', sharing = 'public')


In [83]:
k = 10
a = res.iloc[1:k].copy()
b = res.iloc[1:k].copy()
c = res.iloc[1:k].copy()
d = res.iloc[1:k].copy()

e = pd.concat([a,b,c,d], axis = 0)

In [84]:
e2 = e.to_crs(epsg = 5179)
e2['grp_rn'] = e2.groupby(['sgg_nm']).cumcount() + 1
e2.sort_values(by = ['sgg_nm'])

n = e2.shape[0]
cnt = 99999
width_m = 500
height_m = 500
threshold = 250

# random number concept
# 1. 무수히 많은 random number generation
# 2. 이 중 가로/세로 거리가 250m 이상인 점들만을 도출
np.random.seed(1575)
rn = np.random.rand(cnt)
rn = rn[rn * width_m > threshold][:n] 
width_rn = rn * width_m

rn = np.random.rand(cnt)
rn = rn[rn * height_m > threshold][:n]
height_rn = rn * height_m

width_idx = np.where(np.random.rand(n) < 0.5, -1, 1)
height_idx = np.where(np.random.rand(n) < 0.5, -1, 1)

In [85]:
#e2['width_move'] = width_rn * width_idx
#e2['height_move'] = height_rn * height_idx

e2['x_utmk'] = e2.ptr_wkt.x + width_rn * width_idx
e2['y_utmk'] = e2.ptr_wkt.y + height_rn * height_idx

e2_res = gpd.GeoDataFrame(e2[['sgg_nm', 'x_utmk', 'y_utmk']],geometry = [Point(x,y) for x,y in zip(e2.x_utmk,e2.y_utmk)],crs = 5179)
e2_res = e2_res.to_crs(epsg = 4326)

e2_res['lon'] = e2_res.geometry.x
e2_res['lat'] = e2_res.geometry.y

In [90]:
fig = px.scatter_mapbox(e2_res,lat = 'lat',lon = 'lon',hover_name = 'sgg_nm',mapbox_style = 'open-street-map')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [87]:
e2_res.sort_values(by=['sgg_nm'])

,sgg_nm,x_utmk,y_utmk,geometry,lon,lat
6,성남시 분당구,965511.145347,1.930914e+06,POINT (127.11045 37.37666),127.110451,37.376657
6,성남시 분당구,965460.152946,1.931627e+06,POINT (127.10984 37.38309),127.109842,37.383087
6,성남시 분당구,964836.908758,1.930730e+06,POINT (127.10284 37.37498),127.102845,37.374978
6,성남시 분당구,965551.209757,1.931517e+06,POINT (127.11088 37.38210),127.110876,37.382097
4,성남시 수정구,965362.434741,1.937788e+06,POINT (127.10845 37.43861),127.108449,37.438614
4,성남시 수정구,965248.132604,1.936983e+06,POINT (127.10720 37.43135),127.107195,37.431351
4,성남시 수정구,965467.141298,1.937677e+06,POINT (127.10964 37.43762),127.109638,37.437620
4,성남시 수정구,965436.509459,1.937047e+06,POINT (127.10932 37.43194),127.109321,37.431941
5,성남시 중원구,970682.951595,1.937670e+06,POINT (127.16860 37.43773),127.168597,37.437732
5,성남시 중원구,970713.094275,1.936848e+06,POINT (127.16897 37.43033),127.168971,37.430330


In [89]:
e2.sort_values(by=['sgg_nm'])

,sgg_cd,sgg_nm,ptr_wkt,lon,lat,grp_rn,x_utmk,y_utmk
6,41135,성남시 분당구,POINT (965123.927 1931208.643),127.106064,37.379302,1,965511.145347,1.930914e+06
6,41135,성남시 분당구,POINT (965123.927 1931208.643),127.106064,37.379302,4,965460.152946,1.931627e+06
6,41135,성남시 분당구,POINT (965123.927 1931208.643),127.106064,37.379302,2,964836.908758,1.930730e+06
6,41135,성남시 분당구,POINT (965123.927 1931208.643),127.106064,37.379302,3,965551.209757,1.931517e+06
4,41131,성남시 수정구,POINT (964980.062 1937407.071),127.104145,37.435166,2,965362.434741,1.937788e+06
4,41131,성남시 수정구,POINT (964980.062 1937407.071),127.104145,37.435166,3,965248.132604,1.936983e+06
4,41131,성남시 수정구,POINT (964980.062 1937407.071),127.104145,37.435166,1,965467.141298,1.937677e+06
4,41131,성남시 수정구,POINT (964980.062 1937407.071),127.104145,37.435166,4,965436.509459,1.937047e+06
5,41133,성남시 중원구,POINT (970266.576 1937194.278),127.163910,37.433433,3,970682.951595,1.937670e+06
5,41133,성남시 중원구,POINT (970266.576 1937194.278),127.163910,37.433433,1,970713.094275,1.936848e+06
